# ETL

# Libraries

In [2]:
import os
import ast

import numpy as np
import pandas as pd

# Extract

Los dos datasets se descargan de esta dirección:

https://drive.google.com/drive/folders/1X_LdCoGTHJDbD28_dJTxaD4fVuQC9Wt5

Los datasets deben estar en una carpeta llamada `raw` ubicada en la raiz del proyecto.

In [3]:
ruta_raw = os.path.abspath('../raw/')

## Extract `movies`

In [4]:
movies = pd.read_csv(
    os.path.join(
        ruta_raw, 'movies_dataset.csv'))

C:\Users\franc\AppData\Local\Temp\ipykernel_20768\2227159362.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(


In [5]:
movies.head().T

,0,1,2,3,4
adult,False,False,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,"{'id': 96871, 'name': 'Father of the Bride Col..."
budget,30000000,65000000,0,16000000,0
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'id': 35, 'name': 'Comedy'}]"
homepage,http://toystory.disney.com/toy-story,NaN,NaN,NaN,NaN
id,862,8844,15602,31357,11862
imdb_id,tt0114709,tt0113497,tt0113228,tt0114885,tt0113041
original_language,en,en,en,en,en
original_title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...,"Cheated on, mistreated and stepped on, the wom...",Just when George Banks has recovered from his ...


## Extract `credits`

In [6]:
credits_df = pd.read_csv(
    os.path.join(
        ruta_raw, 'credits.csv'))

In [7]:
credits_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


# Transform

Transformaciones a realizar:

- Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

- Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

- Los valores nulos del campo release date deben eliminarse.

- De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

- Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

- Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

## Transform movies

In [8]:
movies.head(1).T

,0
adult,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ..."
budget,30000000
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
homepage,http://toystory.disney.com/toy-story
id,862
imdb_id,tt0114709
original_language,en
original_title,Toy Story
overview,"Led by Woody, Andy's toys live happily in his ..."


### Unnesting

#### belongs_to_collection

In [9]:
belongs_to_collection = movies[['id', 'belongs_to_collection']].copy()

movies = movies.drop(columns=['belongs_to_collection'])

In [10]:
belongs_to_collection.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     45466 non-null  object
 1   belongs_to_collection  4494 non-null   object
dtypes: object(2)
memory usage: 710.5+ KB


In [11]:
belongs_to_collection.head(5)

,id,belongs_to_collection
0,862,"{'id': 10194, 'name': 'Toy Story Collection', ..."
1,8844,NaN
2,15602,"{'id': 119050, 'name': 'Grumpy Old Men Collect..."
3,31357,NaN
4,11862,"{'id': 96871, 'name': 'Father of the Bride Col..."


In [12]:
belongs_to_collection.rename(
    columns={
        'id': 'movie_id',
        'belongs_to_collection': 'collection_name'
    },
    inplace=True)

In [13]:
belongs_to_collection.head(5)

,movie_id,collection_name
0,862,"{'id': 10194, 'name': 'Toy Story Collection', ..."
1,8844,NaN
2,15602,"{'id': 119050, 'name': 'Grumpy Old Men Collect..."
3,31357,NaN
4,11862,"{'id': 96871, 'name': 'Father of the Bride Col..."


In [14]:
belongs_to_collection['collection_name'] = belongs_to_collection['collection_name'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)

new_columns = pd.json_normalize(belongs_to_collection['collection_name'])

belongs_to_collection = belongs_to_collection.join(new_columns)

belongs_to_collection.drop(columns=['collection_name'], inplace=True)

belongs_to_collection.rename(
    columns={
        'id': 'collection_id',
        'name': 'collection_name'
    },
    inplace=True)

In [15]:
belongs_to_collection.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         45466 non-null  object 
 1   collection_id    4491 non-null   float64
 2   collection_name  4491 non-null   object 
 3   poster_path      3948 non-null   object 
 4   backdrop_path    3263 non-null   object 
dtypes: float64(1), object(4)
memory usage: 1.7+ MB


In [16]:
belongs_to_collection.head()

,movie_id,collection_id,collection_name,poster_path,backdrop_path
0,862,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,8844,NaN,NaN,NaN,NaN
2,15602,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,31357,NaN,NaN,NaN,NaN
4,11862,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg


In [17]:
belongs_to_collection['collection_id'].value_counts()

collection_id
415931.0    29
421566.0    27
645.0       26
96887.0     26
37261.0     25
            ..
463112.0     1
474012.0     1
284700.0     1
142686.0     1
267551.0     1
Name: count, Length: 1695, dtype: int64

#### genres

In [18]:
genres = movies[['id', 'genres']].copy()

movies = movies.drop(columns=['genres'])

In [19]:
genres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      45466 non-null  object
 1   genres  45466 non-null  object
dtypes: object(2)
memory usage: 710.5+ KB


In [20]:
genres.head(5)

,id,genres
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,"[{'id': 35, 'name': 'Comedy'}]"


In [21]:
genres.rename(
    columns={
        'id': 'movie_id'
    },
    inplace=True)

In [22]:
genres.head(5)

,movie_id,genres
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,"[{'id': 35, 'name': 'Comedy'}]"


In [23]:
genres['genres'] = genres['genres'].apply(lambda x: ast.literal_eval(str(x)) if isinstance(x, str) and pd.notna(x) else x)

exploded = genres.explode('genres')

new_columns = pd.json_normalize(exploded['genres'])

genres = exploded.drop(columns=['genres']).reset_index(drop=True).join(new_columns)

In [24]:
genres.head(5)

,movie_id,id,name
0,862,16.0,Animation
1,862,35.0,Comedy
2,862,10751.0,Family
3,8844,12.0,Adventure
4,8844,14.0,Fantasy


In [25]:
genres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93548 entries, 0 to 93547
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   movie_id  93548 non-null  object 
 1   id        91106 non-null  float64
 2   name      91106 non-null  object 
dtypes: float64(1), object(2)
memory usage: 2.1+ MB


In [26]:
genres.rename(
    columns={
        'id': 'genre_id',
        'name': 'genre_name'
    },
    inplace=True)

In [27]:
genres.columns

Index(['movie_id', 'genre_id', 'genre_name'], dtype='object')

In [28]:
genres['genre_id'].value_counts()

genre_id
18.0       20265
35.0       13182
53.0        7624
10749.0     6735
28.0        6596
27.0        4673
80.0        4307
99.0        3932
12.0        3496
878.0       3049
10751.0     2770
9648.0      2467
14.0        2313
16.0        1935
10769.0     1622
10402.0     1598
36.0        1398
10752.0     1323
37.0        1042
10770.0      767
11176.0        1
11602.0        1
29812.0        1
2883.0         1
7759.0         1
7760.0         1
7761.0         1
33751.0        1
17161.0        1
18012.0        1
18013.0        1
23822.0        1
Name: count, dtype: int64

#### production_companies

In [29]:
production_companies = movies[['id', 'production_companies']].copy()

movies = movies.drop(columns=['production_companies'])

In [30]:
production_companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    45466 non-null  object
 1   production_companies  45463 non-null  object
dtypes: object(2)
memory usage: 710.5+ KB


In [31]:
production_companies.head(5)

,id,production_companies
0,862,"[{'name': 'Pixar Animation Studios', 'id': 3}]"
1,8844,"[{'name': 'TriStar Pictures', 'id': 559}, {'na..."
2,15602,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'..."
3,31357,[{'name': 'Twentieth Century Fox Film Corporat...
4,11862,"[{'name': 'Sandollar Productions', 'id': 5842}..."


In [32]:
production_companies.rename(
    columns={
        'id': 'movie_id'
    },
    inplace=True)

In [33]:
production_companies.columns

Index(['movie_id', 'production_companies'], dtype='object')

In [34]:
production_companies['production_companies'] = production_companies['production_companies'].apply(lambda x: ast.literal_eval(str(x)) if isinstance(x, str) and pd.notna(x) else x)

exploded = production_companies.explode('production_companies')

new_columns = pd.json_normalize(exploded['production_companies'])

production_companies = exploded.drop(columns=['production_companies']).reset_index(drop=True).join(new_columns)

In [35]:
production_companies.head(5)

,movie_id,name,id
0,862,Pixar Animation Studios,3.0
1,8844,TriStar Pictures,559.0
2,8844,Teitler Film,2550.0
3,8844,Interscope Communications,10201.0
4,15602,Warner Bros.,6194.0


In [36]:
production_companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82426 entries, 0 to 82425
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   movie_id  82426 non-null  object 
 1   name      70545 non-null  object 
 2   id        70545 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.9+ MB


In [37]:
production_companies.rename(
    columns={
        'id': 'company_id', 
        'name': 'company_name'
    },
    inplace=True)

In [38]:
production_companies.columns

Index(['movie_id', 'company_name', 'company_id'], dtype='object')

In [39]:
production_companies['company_name'].value_counts()

company_name
Warner Bros.                              1250
Metro-Goldwyn-Mayer (MGM)                 1076
Paramount Pictures                        1003
Twentieth Century Fox Film Corporation     836
Universal Pictures                         830
                                          ... 
Inside Track 1                               1
Film Council Premiere Fund                   1
Untitled 03                                  1
absolutely productions                       1
Yermoliev                                    1
Name: count, Length: 23537, dtype: int64

#### production_countries

In [40]:
production_countries = movies[['id', 'production_countries']].copy()

movies = movies.drop(columns=['production_countries'])

In [41]:
production_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    45466 non-null  object
 1   production_countries  45463 non-null  object
dtypes: object(2)
memory usage: 710.5+ KB


In [42]:
production_countries.head(5)

,id,production_countries
0,862,"[{'iso_3166_1': 'US', 'name': 'United States o..."
1,8844,"[{'iso_3166_1': 'US', 'name': 'United States o..."
2,15602,"[{'iso_3166_1': 'US', 'name': 'United States o..."
3,31357,"[{'iso_3166_1': 'US', 'name': 'United States o..."
4,11862,"[{'iso_3166_1': 'US', 'name': 'United States o..."


In [43]:
production_countries.rename(
    columns={
        'id': 'movie_id'
    },
    inplace=True)

In [44]:
production_countries.columns

Index(['movie_id', 'production_countries'], dtype='object')

In [45]:
production_countries['production_countries'] = production_countries['production_countries'].apply(lambda x: ast.literal_eval(str(x)) if isinstance(x, str) and pd.notna(x) else x)

exploded = production_countries.explode('production_countries')

new_columns = pd.json_normalize(exploded['production_countries'])

production_countries = exploded.drop(columns=['production_countries']).reset_index(drop=True).join(new_columns)

In [46]:
production_countries.head(5)

,movie_id,iso_3166_1,name
0,862,US,United States of America
1,8844,US,United States of America
2,15602,US,United States of America
3,31357,US,United States of America
4,11862,US,United States of America


In [47]:
production_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55711 entries, 0 to 55710
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   movie_id    55711 non-null  object
 1   iso_3166_1  49423 non-null  object
 2   name        49423 non-null  object
dtypes: object(3)
memory usage: 1.3+ MB


In [48]:
production_countries.rename(
    columns={
        'name': 'country_name'
    },
    inplace=True)

In [49]:
production_countries.columns

Index(['movie_id', 'iso_3166_1', 'country_name'], dtype='object')

In [50]:
production_countries['country_name'].value_counts()

country_name
United States of America    21153
United Kingdom               4094
France                       3940
Germany                      2254
Italy                        2169
                            ...  
Antarctica                      1
Gibraltar                       1
Brunei Darussalam               1
Honduras                        1
Guinea                          1
Name: count, Length: 160, dtype: int64

#### spoken_languages

In [51]:
spoken_languages = movies[['id', 'spoken_languages']].copy()

movies = movies.drop(columns=['spoken_languages'])

In [52]:
spoken_languages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                45466 non-null  object
 1   spoken_languages  45460 non-null  object
dtypes: object(2)
memory usage: 710.5+ KB


In [53]:
spoken_languages.head(5)

,id,spoken_languages
0,862,"[{'iso_639_1': 'en', 'name': 'English'}]"
1,8844,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso..."
2,15602,"[{'iso_639_1': 'en', 'name': 'English'}]"
3,31357,"[{'iso_639_1': 'en', 'name': 'English'}]"
4,11862,"[{'iso_639_1': 'en', 'name': 'English'}]"


In [54]:
spoken_languages.rename(
    columns={
        'id': 'movie_id'
    },
    inplace=True)

In [55]:
spoken_languages.columns

Index(['movie_id', 'spoken_languages'], dtype='object')

In [56]:
spoken_languages['spoken_languages'] = spoken_languages['spoken_languages'].apply(lambda x: ast.literal_eval(str(x)) if isinstance(x, str) and pd.notna(x) else x)

exploded = spoken_languages.explode('spoken_languages')

new_columns = pd.json_normalize(exploded['spoken_languages'])

spoken_languages = exploded.drop(columns=['spoken_languages']).reset_index(drop=True).join(new_columns)

In [57]:
spoken_languages.head(5)

,movie_id,iso_639_1,name
0,862,en,English
1,8844,en,English
2,8844,fr,Français
3,15602,en,English
4,31357,en,English


In [58]:
spoken_languages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57135 entries, 0 to 57134
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   movie_id   57135 non-null  object
 1   iso_639_1  53300 non-null  object
 2   name       53300 non-null  object
dtypes: object(3)
memory usage: 1.3+ MB


In [59]:
spoken_languages.rename(
    columns={
        'name': 'language_name'
    },
    inplace=True)

In [60]:
spoken_languages.columns

Index(['movie_id', 'iso_639_1', 'language_name'], dtype='object')

In [61]:
spoken_languages['language_name'].value_counts()

language_name
English            28745
Français            4196
Deutsch             2625
Español             2413
Italiano            2367
                   ...  
Kinyarwanda            3
беларуская мова        2
Fulfulde               2
ozbek                  2
Hausa                  1
Name: count, Length: 75, dtype: int64

In [62]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              45466 non-null  object 
 1   budget             45466 non-null  object 
 2   homepage           7782 non-null   object 
 3   id                 45466 non-null  object 
 4   imdb_id            45449 non-null  object 
 5   original_language  45455 non-null  object 
 6   original_title     45466 non-null  object 
 7   overview           44512 non-null  object 
 8   popularity         45461 non-null  object 
 9   poster_path        45080 non-null  object 
 10  release_date       45379 non-null  object 
 11  revenue            45460 non-null  float64
 12  runtime            45203 non-null  float64
 13  status             45379 non-null  object 
 14  tagline            20412 non-null  object 
 15  title              45460 non-null  object 
 16  video              454

### Fill in the revenue and budget fields with 0

In [63]:
movies['revenue'].value_counts()

revenue
0.0           38052
12000000.0       20
10000000.0       19
11000000.0       19
2000000.0        18
              ...  
320395.0          1
50656.0           1
1328612.0         1
1268793.0         1
103504.0          1
Name: count, Length: 6863, dtype: int64

In [64]:
movies['budget'].value_counts()

budget
0           36573
5000000       286
10000000      259
20000000      243
2000000       242
            ...  
12200000        1
65700           1
64200           1
707503          1
8620000         1
Name: count, Length: 1226, dtype: int64

In [65]:
display(movies['revenue'].isna().sum())
display(movies['budget'].isna().sum())

np.int64(6)

np.int64(0)

In [66]:
movies[['revenue', 'budget']] = movies[['revenue', 'budget']].fillna(0)

display(movies['revenue'].isna().sum())
display(movies['budget'].isna().sum())

np.int64(0)

np.int64(0)

### Remove null values ​​from the release_date field

In [67]:
movies['release_date'].isna().sum()

np.int64(87)

In [68]:
movies = movies.dropna(subset=['release_date'])

In [69]:
movies['release_date'].isna().sum()

np.int64(0)

### Format YYYY-mm-dd and create the release_year column

In [70]:
movies['release_date'].head(5)

0    1995-10-30
1    1995-12-15
2    1995-12-22
3    1995-12-22
4    1995-02-10
Name: release_date, dtype: object

In [71]:
movies['release_date'] = pd.to_datetime(movies['release_date'], format='%Y-%m-%d', errors='coerce')

In [72]:
movies['release_date'].isna().sum()

np.int64(3)

In [73]:
movies[movies['release_date'].isna()]

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Beware Of Frost Bites,Avalanche Sharks,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
movies = movies.dropna(subset=['release_date'])

movies['release_date'].isna().sum()

np.int64(0)

In [75]:
movies['release_year'] = pd.to_datetime(movies['release_date']).dt.year

movies['release_year'].head(5)

0    1995
1    1995
2    1995
3    1995
4    1995
Name: release_year, dtype: int32

### Create return column

In [76]:
movies['revenue'] = pd.to_numeric(movies['revenue']).astype(int)
movies['budget'] = pd.to_numeric(movies['budget']).astype(int)

movies['return'] = np.where((movies['revenue'] == 0) | (movies['budget'] == 0), 0, movies['revenue'] // movies['budget']).astype(int)

display(movies[['revenue', 'budget', 'return']].head(5))
print()
display(movies['return'].value_counts())

,revenue,budget,return
0,373554033,30000000,12
1,262797249,65000000,4
2,0,0,0
3,81452156,16000000,5
4,76578911,0,0


return
0       41600
1        1021
2         810
3         497
4         341
        ...  
426         1
69          1
2216        1
50          1
155         1
Name: count, Length: 113, dtype: int64

### Delete unnecessary columns

In [77]:
movies.columns

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'release_year', 'return'],
      dtype='object')

In [78]:
movies = movies.drop(columns=["video", "imdb_id", "adult", "original_title", "poster_path", "homepage"])

In [79]:
movies.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'release_year', 'return'],
      dtype='object')